In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
import tensorflow as tf
from sklearn.decomposition import PCA
import os

2024-01-21 22:48:02.370359: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-21 22:48:02.402645: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-21 22:48:02.402671: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-21 22:48:02.403647: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-21 22:48:02.408592: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-21 22:48:02.408919: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
def get_image_paths(directory):
    image_paths = []
    valid_extensions = ['.jpg', '.jpeg', '.png', '.gif']  # Add more extensions if needed

    for filename in os.listdir(directory):
        if any(filename.lower().endswith(ext) for ext in valid_extensions):
            image_paths.append(os.path.join(directory, filename))

    return image_paths

In [37]:
def get_vector(img_path): 
    img = image.load_img(img_path, target_size=(70,70))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = model.predict(x)

    return features.flatten()

In [38]:
import tensorflow as tf


# Load pre-trained VGG16 model
model = VGG16(weights='imagenet', include_top=False)

# Load and preprocess your dataset images
# Assuming 'image_paths' is a list of paths to your images
embeddings = []
image_paths = get_image_paths("/home/mahdi/Projects/FairnessLens/FERET/colorferet/dvd1/colored/")
for img_path in image_paths:
    embeddings.append(get_vector(img_path))

1/1 [==============================] - 0s 23ms/step


In [40]:
len(embeddings[1])

2048

In [41]:
mean_vector = np.mean(embeddings, axis=0)
mean_vector

array([ 0.       ,  0.       ,  0.       , ...,  0.2136522,  0.       ,
       10.836747 ], dtype=float32)

In [42]:
covariance_matrix = np.cov(embeddings, rowvar=False)
covariance_matrix

array([[  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [  0.        ,   0.        ,   0.        , ...,   3.81406282,
          0.        ,   0.47289027],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.47289027,
          0.        , 145.60973427]])

In [43]:
covariance_matrix.shape

(2048, 2048)

In [28]:
# sample_vector = get_vector("/home/mahdi/Projects/FairnessLens/Gateway/results/feret/1_1_20231227205821214092.png")
sample_vector = get_vector("/home/mahdi/Projects/FairnessLens/FERET/colorferet/dvd1/colored/00043_931230_fa_converted_final.png")

1/1 [==============================] - 0s 24ms/step


In [29]:
sample_vector

array([ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
       53.408577 ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       , 16.127626 ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.    

In [30]:
difference_vector = sample_vector - mean_vector

In [35]:
covariance_matrix_regularized = covariance_matrix + 1e-5 * np.eye(covariance_matrix.shape[0])
covariance_matrix_inverse = np.linalg.inv(covariance_matrix_regularized)

In [36]:
result = np.dot(np.dot(difference_vector.T, covariance_matrix_regularized), difference_vector)
result

13556955.633879256